In [20]:
!curl -s -X DELETE http://localhost:9200/sqe_metrics/

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [sqe_metrics]","index":"sqe_metrics","resource.id":"sqe_metrics","resource.type":"index_or_alias","index_uuid":"_na_"}],"type":"index_not_found_exception","reason":"no such index [sqe_metrics]","index":"sqe_metrics","resource.id":"sqe_metrics","resource.type":"index_or_alias","index_uuid":"_na_"},"status":404}

In [174]:
# Hack needed due to not high enough disk space locally
# See reference at bottom of Notebook

# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

{"acknowledged":true,"persistent":{},"transient":{"cluster":{"routing":{"allocation":{"disk":{"threshold_enabled":"false"}}}}}}

In [21]:
import time
import json
import random
import uuid
import pandas as pd

from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200", use_ssl=False)

In [22]:
df = pd.read_csv("metrics.csv")

In [23]:
df.head()

,datetime,search_config,query_set_id,query,metric,value
0,2024-07-01,research_1,sampled_queries,00g 目立たない,dcg,1.633380
1,2024-07-01,research_1,sampled_queries,1 & 1/4 pvp water suction hose without couplin...,dcg,0.117057
2,2024-07-01,research_1,sampled_queries,10 and 8 yr old. make up without the nieal plush,dcg,1.630930
3,2024-07-01,research_1,sampled_queries,11' x 11' patio umbrella square without led li...,dcg,1.081171
4,2024-07-01,research_1,sampled_queries,23x10.5 tractor intertubes,dcg,1.000000


In [24]:
bulk = []

for ix, metric in df.iterrows():
    metric_dict = {
        "datetime": metric["datetime"],
        "search_config": metric["search_config"],
        "query_set_id": metric["query_set_id"],
        "query": metric["query"],
        "metric": metric["metric"],
        "value": metric["value"],
    }
    event_id = str(uuid.uuid4())
    bulk.append(
        {"index": {"_index": "sqe_metrics", "_id": event_id}}
    )
    bulk.append(metric_dict)

response = client.bulk(bulk)

In [25]:
df.query_set_id.size

36960

In [26]:
!curl -s http://localhost:9200/sqe_metrics/_count | jq

{
  "count": 36248,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  }
}


In [27]:
!curl -s -XGET http://localhost:9200/sqe_metrics/_search -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
    "match": { \
      "query": "4t camo sweat shirt" \
    } \
  } \
}' | jq


{
  "took": 19,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 672,
      "relation": "eq"
    },
    "max_score": 18.596354,
    "hits": [
      {
        "_index": "sqe_metrics",
        "_id": "772b1227-6f2c-4362-b27b-7b2a1ac50547",
        "_score": 18.596354,
        "_source": {
          "datetime": "2024-07-01",
          "search_config": "research_1",
          "query_set_id": "sampled_queries",
          "query": "4t camo sweat shirt",
          "metric": "dcg",
          "value": 0.0717734625362931
        }
      },
      {
        "_index": "sqe_metrics",
        "_id": "675e18e0-3b7c-470f-b43f-85e4ec93472f",
        "_score": 18.596354,
        "_source": {
          "datetime": "2024-06-30",
          "search_config": "research_1",
          "query_set_id": "sampled_queries",
          "query": "4t camo sweat shirt",
          "metric": "dcg",
          "value": 0.

In [28]:
# Hacky, disk space checks need to be disabled on my system to allow Dashboards to work
#
# https://stackoverflow.com/a/57675654

# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'
# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'

{"acknowledged":true,"persistent":{},"transient":{"cluster":{"routing":{"allocation":{"disk":{"threshold_enabled":"false"}}}}}}{"acknowledged":true}